Question 1: Please help me find out the license thing

### first approach 

In [11]:
%time
# PDF to HTML
# http://macappstore.org/pdf2htmlex/
# https://github.com/coolwanglu/pdf2htmlEX/wiki/Command-Line-Options
import os
echo = "pdf2htmlEX --zoom 1.3"
for filename in os.listdir("pdf_files"):
    if filename.endswith("pdf"):
        print(filename)
        #os.system(echo+" "+filename)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.11 µs
CMS_2014_0115_12304.pdf
CMS_2014_0115_58152.pdf
CMS_2014_0115_58434.pdf
CMS_2014_0115_56380.pdf
CMS_2014_0115_5633.pdf
CMS_2014_0115_52496.pdf
CMS_2014_0115_58186.pdf
CMS_2014_0115_52483.pdf
CMS_2014_0115_13187.pdf
CMS_2014_0115_12528.pdf
CMS_2014_0115_57882.pdf
CMS_2014_0115_9434.pdf
CMS_2014_0115_58168.pdf
CMS_2014_0115_13182.pdf
CMS_2014_0115_12505.pdf
CMS_2014_0115_58424.pdf
CMS_2014_0115_51400.pdf
CMS_2014_0115_58430.pdf
CMS_2014_0115_53363.pdf
CMS_2014_0115_55706.pdf
CMS_2014_0115_13035.pdf
CMS_2014_0115_58008.pdf
CMS_2014_0115_12895.pdf
CMS_2014_0115_13235.pdf
CMS_2014_0115_53188.pdf
CMS_2014_0115_58432.pdf
CMS_2014_0115_56218.pdf
CMS_2014_0115_54369.pdf
CMS_2014_0115_51921.pdf
CMS_2014_0115_54291.pdf
CMS_2014_0115_0059.pdf
CMS_2014_0115_13440.pdf
CMS_2014_0115_12762.pdf
CMS_2014_0115_56336.pdf
CMS_2014_0115_52959.pdf
CMS_2014_0115_56256.pdf
CMS_2014_0115_13244.pdf
CMS_2014_0115_53258.pdf
CMS_2014_0115_6167.pdf
CMS_

## second approach (current one)

In [1]:
%time
# PDF to HTML
# https://sourceforge.net/projects/pdftohtml/
# https://www.experts-exchange.com/videos/59892/Xpdf-PDFtoHTML-Command-Line-Utility-to-Convert-a-PDF-File-to-HTML.html
# https://www.xpdfreader.com/download.html

# make sure to have a folder named "pdf_files" with the pdfs
# make sure to have an empty folder named "html_files"
import os
echo = "pdftohtml"
for filename in os.listdir("pdf_files"):
    if filename.endswith("pdf"):
        print(filename)
        file_pdf = "pdf_files/" + filename
        file_html = "html_files/" + filename
        #os.system(echo+" "+file_pdf+" "+file_html) 

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.78 µs
CMS_2014_0115_12304.pdf
CMS_2014_0115_58152.pdf
CMS_2014_0115_58434.pdf
CMS_2014_0115_56380.pdf
CMS_2014_0115_5633.pdf
CMS_2014_0115_52496.pdf
CMS_2014_0115_58186.pdf
CMS_2014_0115_52483.pdf
CMS_2014_0115_13187.pdf
CMS_2014_0115_12528.pdf
CMS_2014_0115_57882.pdf
CMS_2014_0115_9434.pdf
CMS_2014_0115_58168.pdf
CMS_2014_0115_13182.pdf
CMS_2014_0115_12505.pdf
CMS_2014_0115_58424.pdf
CMS_2014_0115_51400.pdf
CMS_2014_0115_58430.pdf
CMS_2014_0115_53363.pdf
CMS_2014_0115_55706.pdf
CMS_2014_0115_13035.pdf
CMS_2014_0115_58008.pdf
CMS_2014_0115_12895.pdf
CMS_2014_0115_13235.pdf
CMS_2014_0115_53188.pdf
CMS_2014_0115_58432.pdf
CMS_2014_0115_56218.pdf
CMS_2014_0115_54369.pdf
CMS_2014_0115_51921.pdf
CMS_2014_0115_54291.pdf
CMS_2014_0115_0059.pdf
CMS_2014_0115_13440.pdf
CMS_2014_0115_12762.pdf
CMS_2014_0115_56336.pdf
CMS_2014_0115_52959.pdf
CMS_2014_0115_56256.pdf
CMS_2014_0115_13244.pdf
CMS_2014_0115_53258.pdf
CMS_2014_0115_6167.pdf
CMS_

## creates table of features

In [2]:
import codecs
from bs4 import BeautifulSoup
import cssutils
import pandas as pd
import matplotlib.pyplot as plt
import os

In [3]:
def style_dict(style_tem):
    style_tem = style_tem[:-1]
    style_dic = dict(item.split(":") for item in style_tem.split(";"))
    return style_dic

In [4]:
def css_style(soup):
    selectors = {}
    for styles in soup.select('style'):
        css = cssutils.parseString(styles.encode_contents())
        for rule in css:
            if rule.type == rule.STYLE_RULE:
                style = rule.selectorText
                selectors[style] = {}
                for item in rule.style:
                    propertyname = item.name
                    value = item.value
                    selectors[style][propertyname] = value

    dict_css = {key.strip('#'): item for key, item in selectors.items()}
    return dict_css

In [5]:
def extract_features(path):
    
    with open(path) as raw_resuls:
        results = BeautifulSoup(raw_resuls, 'lxml')
    
    dict_css = css_style(results)
    lines = []
    text = []
    for element in results.find_all("body"):
        for stat in element.find_all("div"):
            data={}
            # gets the info from <div>
            div_dic = stat.attrs
            data.update(div_dic)

            # gets info from <span>
            span_dic = stat.find('span').attrs
            data.update(span_dic)

            # gets the features from style in <div>
            style_ = span_dic.get('style')
            style_span = style_dict(style_)
            data.update(style_span)

            # gets features of the font from css 
            font = span_dic.get('id')
            temp = dict_css.get(font)
            data.update(temp)

            # gets the features from style in <div>
            style_ = div_dic.get('style')
            style_temp = style_dict(style_)
            data.update(style_temp)
            
            d = pd.DataFrame.from_dict(data)
            
            text.append(stat.text)
            #print(stat.text)
            lines.append(d)

    dataframe = pd.concat(lines, axis=0).reset_index(drop=1)
    dataframe["text"] = pd.Series( (v for v in text))
    
    return dataframe

### reads all html files

In [6]:
%time
features = []
for root, dirs, files in os.walk("html_files"):
    for file in files:
        if file.endswith(".html"):
            if file != "index.html":
                try:
                    file_path = os.path.join(root, file)
                    print(file_path+ " found") 
                    df = extract_features(file_path)
                    df = df.assign(page=file)
                    df = df.assign(document=root)
                    features.append(df)
                except:
                    print(file_path+" NOT found - exception thrown")

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 7.87 µs
html_files/CMS_2014_0115_12304.pdf/page1.html found
html_files/CMS_2014_0115_12304.pdf/page7.html found
html_files/CMS_2014_0115_12304.pdf/page6.html found
html_files/CMS_2014_0115_12304.pdf/page5.html found
html_files/CMS_2014_0115_12304.pdf/page4.html found
html_files/CMS_2014_0115_12304.pdf/page8.html found
html_files/CMS_2014_0115_12304.pdf/page3.html found
html_files/CMS_2014_0115_12304.pdf/page2.html found
html_files/CMS_2014_0115_58152.pdf/page18.html found
html_files/CMS_2014_0115_58152.pdf/page1.html found
html_files/CMS_2014_0115_58152.pdf/page14.html found
html_files/CMS_2014_0115_58152.pdf/page15.html found
html_files/CMS_2014_0115_58152.pdf/page19.html found
html_files/CMS_2014_0115_58152.pdf/page12.html found
html_files/CMS_2014_0115_58152.pdf/page7.html found
html_files/CMS_2014_0115_58152.pdf/page6.html found
html_files/CMS_2014_0115_58152.pdf/page13.html found
html_files/CMS_2014_0115_58152.pdf/page9.html 

/Users/msyeom/anaconda3/envs/pdf/lib/python3.7/site-packages/ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



html_files/CMS_2014_0115_5633.pdf/page3.html found
html_files/CMS_2014_0115_5633.pdf/page2.html found
html_files/CMS_2014_0115_52496.pdf/page1.html found
html_files/CMS_2014_0115_52496.pdf/page7.html found
html_files/CMS_2014_0115_52496.pdf/page6.html found
html_files/CMS_2014_0115_52496.pdf/page9.html found
html_files/CMS_2014_0115_52496.pdf/page10.html found
html_files/CMS_2014_0115_52496.pdf/page5.html found
html_files/CMS_2014_0115_52496.pdf/page4.html found
html_files/CMS_2014_0115_52496.pdf/page11.html found
html_files/CMS_2014_0115_52496.pdf/page8.html found
html_files/CMS_2014_0115_52496.pdf/page3.html found
html_files/CMS_2014_0115_52496.pdf/page2.html found
html_files/CMS_2014_0115_58186.pdf/page18.html found
html_files/CMS_2014_0115_58186.pdf/page1.html found
html_files/CMS_2014_0115_58186.pdf/page22.html found
html_files/CMS_2014_0115_58186.pdf/page14.html found
html_files/CMS_2014_0115_58186.pdf/page15.html found
html_files/CMS_2014_0115_58186.pdf/page23.html found
html_fi

html_files/CMS_2014_0115_55706.pdf/page2.html found
html_files/CMS_2014_0115_13035.pdf/page1.html found
html_files/CMS_2014_0115_13035.pdf/page2.html found
html_files/CMS_2014_0115_58008.pdf/page18.html found
html_files/CMS_2014_0115_58008.pdf/page1.html found
html_files/CMS_2014_0115_58008.pdf/page14.html found
html_files/CMS_2014_0115_58008.pdf/page15.html found
html_files/CMS_2014_0115_58008.pdf/page19.html found
html_files/CMS_2014_0115_58008.pdf/page12.html found
html_files/CMS_2014_0115_58008.pdf/page7.html found
html_files/CMS_2014_0115_58008.pdf/page6.html found
html_files/CMS_2014_0115_58008.pdf/page13.html found
html_files/CMS_2014_0115_58008.pdf/page9.html found
html_files/CMS_2014_0115_58008.pdf/page10.html found
html_files/CMS_2014_0115_58008.pdf/page5.html found
html_files/CMS_2014_0115_58008.pdf/page4.html found
html_files/CMS_2014_0115_58008.pdf/page11.html found
html_files/CMS_2014_0115_58008.pdf/page8.html found
html_files/CMS_2014_0115_58008.pdf/page3.html found
html

html_files/CMS_2014_0115_53258.pdf/page5.html found
html_files/CMS_2014_0115_53258.pdf/page4.html found
html_files/CMS_2014_0115_53258.pdf/page3.html found
html_files/CMS_2014_0115_53258.pdf/page2.html found
html_files/CMS_2014_0115_6167.pdf/page1.html found
html_files/CMS_2014_0115_6167.pdf/page5.html found
html_files/CMS_2014_0115_6167.pdf/page4.html found
html_files/CMS_2014_0115_6167.pdf/page3.html found
html_files/CMS_2014_0115_6167.pdf/page2.html found
html_files/CMS_2014_0115_57943.pdf/page1.html found
html_files/CMS_2014_0115_57943.pdf/page14.html found
html_files/CMS_2014_0115_57943.pdf/page15.html found
html_files/CMS_2014_0115_57943.pdf/page12.html found
html_files/CMS_2014_0115_57943.pdf/page7.html found
html_files/CMS_2014_0115_57943.pdf/page6.html found
html_files/CMS_2014_0115_57943.pdf/page13.html found
html_files/CMS_2014_0115_57943.pdf/page9.html found
html_files/CMS_2014_0115_57943.pdf/page10.html found
html_files/CMS_2014_0115_57943.pdf/page5.html found
html_files/C

html_files/CMS_2014_0115_58256.pdf/page7.html found
html_files/CMS_2014_0115_58256.pdf/page6.html found
html_files/CMS_2014_0115_58256.pdf/page13.html found
html_files/CMS_2014_0115_58256.pdf/page9.html found
html_files/CMS_2014_0115_58256.pdf/page10.html found
html_files/CMS_2014_0115_58256.pdf/page5.html found
html_files/CMS_2014_0115_58256.pdf/page4.html found
html_files/CMS_2014_0115_58256.pdf/page11.html found
html_files/CMS_2014_0115_58256.pdf/page8.html found
html_files/CMS_2014_0115_58256.pdf/page3.html found
html_files/CMS_2014_0115_58256.pdf/page2.html found
html_files/CMS_2014_0115_58059.pdf/page1.html found
html_files/CMS_2014_0115_58059.pdf/page12.html found
html_files/CMS_2014_0115_58059.pdf/page7.html found
html_files/CMS_2014_0115_58059.pdf/page6.html found
html_files/CMS_2014_0115_58059.pdf/page9.html found
html_files/CMS_2014_0115_58059.pdf/page10.html found
html_files/CMS_2014_0115_58059.pdf/page5.html found
html_files/CMS_2014_0115_58059.pdf/page4.html found
html_fi

html_files/CMS_2014_0115_13099.pdf/page2.html found
html_files/CMS_2014_0115_58259.pdf/page1.html found
html_files/CMS_2014_0115_58259.pdf/page12.html found
html_files/CMS_2014_0115_58259.pdf/page7.html found
html_files/CMS_2014_0115_58259.pdf/page6.html found
html_files/CMS_2014_0115_58259.pdf/page13.html found
html_files/CMS_2014_0115_58259.pdf/page9.html found
html_files/CMS_2014_0115_58259.pdf/page10.html found
html_files/CMS_2014_0115_58259.pdf/page5.html found
html_files/CMS_2014_0115_58259.pdf/page4.html found
html_files/CMS_2014_0115_58259.pdf/page11.html found
html_files/CMS_2014_0115_58259.pdf/page8.html found
html_files/CMS_2014_0115_58259.pdf/page3.html found
html_files/CMS_2014_0115_58259.pdf/page2.html found
html_files/CMS_2014_0115_52774.pdf/page1.html found
html_files/CMS_2014_0115_52774.pdf/page3.html found
html_files/CMS_2014_0115_52774.pdf/page2.html found
html_files/CMS_2014_0115_9664.pdf/page1.html found
html_files/CMS_2014_0115_9664.pdf/page4.html found
html_files

html_files/CMS_2014_0115_12912.pdf/page7.html found
html_files/CMS_2014_0115_12912.pdf/page6.html found
html_files/CMS_2014_0115_12912.pdf/page5.html found
html_files/CMS_2014_0115_12912.pdf/page4.html found
html_files/CMS_2014_0115_12912.pdf/page3.html found
html_files/CMS_2014_0115_12912.pdf/page2.html found
html_files/CMS_2014_0115_52488.pdf/page1.html found
html_files/CMS_2014_0115_52488.pdf/page12.html found
html_files/CMS_2014_0115_52488.pdf/page7.html found
html_files/CMS_2014_0115_52488.pdf/page6.html found
html_files/CMS_2014_0115_52488.pdf/page9.html found
html_files/CMS_2014_0115_52488.pdf/page10.html found
html_files/CMS_2014_0115_52488.pdf/page5.html found
html_files/CMS_2014_0115_52488.pdf/page4.html found
html_files/CMS_2014_0115_52488.pdf/page11.html found
html_files/CMS_2014_0115_52488.pdf/page8.html found
html_files/CMS_2014_0115_52488.pdf/page3.html found
html_files/CMS_2014_0115_52488.pdf/page2.html found
html_files/CMS_2014_0115_13166.pdf/page1.html found
html_file

html_files/CMS_2014_0115_13028.pdf/page8.html found
html_files/CMS_2014_0115_13028.pdf/page3.html found
html_files/CMS_2014_0115_13028.pdf/page2.html found
html_files/CMS_2014_0115_58001.pdf/page1.html found
html_files/CMS_2014_0115_58001.pdf/page7.html found
html_files/CMS_2014_0115_58001.pdf/page6.html found
html_files/CMS_2014_0115_58001.pdf/page5.html found
html_files/CMS_2014_0115_58001.pdf/page4.html found
html_files/CMS_2014_0115_58001.pdf/page3.html found
html_files/CMS_2014_0115_58001.pdf/page2.html found
html_files/CMS_2014_0115_58017.pdf/page1.html found
html_files/CMS_2014_0115_58017.pdf/page7.html found
html_files/CMS_2014_0115_58017.pdf/page6.html found
html_files/CMS_2014_0115_58017.pdf/page5.html found
html_files/CMS_2014_0115_58017.pdf/page4.html found
html_files/CMS_2014_0115_58017.pdf/page3.html found
html_files/CMS_2014_0115_58017.pdf/page2.html found
html_files/CMS_2014_0115_13162.pdf/page1.html found
html_files/CMS_2014_0115_13162.pdf/page2.html found
html_files/C

In [10]:
os.walk("html_files")

<generator object walk at 0x121e2b228>

In [11]:
df = pd.concat(features, axis=0, sort=False).reset_index(drop=1)
df["Num_Words"] = df["text"].apply(lambda x: len(x.split()))
df["String_Length"] = df["text"].apply(lambda x: len(x))

## Adding text features to the table

In [12]:
df['page']

0        page1.html
1        page1.html
2        page1.html
3        page1.html
4        page1.html
5        page1.html
6        page1.html
7        page1.html
8        page1.html
9        page1.html
10       page1.html
11       page1.html
12       page1.html
13       page1.html
14       page1.html
15       page1.html
16       page1.html
17       page1.html
18       page1.html
19       page1.html
20       page1.html
21       page1.html
22       page1.html
23       page1.html
24       page1.html
25       page1.html
26       page1.html
27       page1.html
28       page1.html
29       page1.html
            ...    
41880    page2.html
41881    page2.html
41882    page2.html
41883    page2.html
41884    page2.html
41885    page2.html
41886    page2.html
41887    page2.html
41888    page2.html
41889    page2.html
41890    page2.html
41891    page2.html
41892    page2.html
41893    page2.html
41894    page2.html
41895    page2.html
41896    page2.html
41897    page2.html
41898    page2.html


In [13]:
df['page_no'] = df['page'].str[:].str.extract('(\d+)').astype(int)

In [14]:
df['page_no']

0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       1
27       1
28       1
29       1
        ..
41880    2
41881    2
41882    2
41883    2
41884    2
41885    2
41886    2
41887    2
41888    2
41889    2
41890    2
41891    2
41892    2
41893    2
41894    2
41895    2
41896    2
41897    2
41898    2
41899    2
41900    2
41901    2
41902    2
41903    2
41904    2
41905    2
41906    2
41907    2
41908    2
41909    2
Name: page_no, Length: 41910, dtype: int64

In [41]:
#df["First_Word"] = df["text"].apply(lambda x: x.split()[0]) # Extract the first word from the string
#df["Last_Word"] = df["text"].apply(lambda x: x.split()[-1]) # Extract the last word from the string

df['Leading_Char_Upper'] = df['text'].str[0].str.isupper()    # Check is starting with uppercase character and assign boolen value
df['Leading_Char_Upper'] = df['Leading_Char_Upper'].map({True: '1', False: '0'})


df['Leading_Numeral'] = df['text'].str[0].str.isdigit() # Check is starting with numeric and assign boolen value 
df['Leading_Numeral'] = df['Leading_Numeral'].map({True: '1', False: '0'})

df['Ends_In_Period'] = df['text'].str.endswith(".") # Check ending with period and assign boolen value 
df['Ends_In_Period'] = df['Ends_In_Period'].map({True: '1', False: '0'})


df['Leading_Number_Period'] = df['text'].str.match("\d+\..*") # Check start with any nuerical combinations followed by ".", ex 1.
df['Leading_Number_Period'] = df['Leading_Number_Period'].map({True: '1', False: '0'})


df['Leading_Char_Period'] = df['text'].str.contains('^[A-Z,a-z]\.+') # Check start with any uppercase or lowercase character  followed by "." , ex A.
df['Leading_Char_Period'] = df['Leading_Char_Period'].map({True: '1', False: '0'})

df['Roman_Period'] = df['text'].str.contains('[V|I|X]\.+') ## Any roman numaral  followed by "."
df['Roman_Period'] = df['Roman_Period'].map({True: '1', False: '0'})

In [42]:
# Extract numbers from values in the " top" column,
# and store them into a new column 'top_px'. e.g. 11px -> 11
df['top_px'] = df[' top'].str.extract('(\d+)').astype(int)

# Extract numbers from values in the " left" column,
# and store them into a new column 'Left_Px'. e.g. 11px -> 11
df['Left_Px'] = df[' left'].str.extract('(\d+)').astype(int)

# Calculate a difference in pixel between lines by assigning
# line space <- pixel at current line minus previous line
df = df.sort_values(by=['document','page_no', 'top_px'], axis='index', ascending=True)
df['LS'] = df.groupby(['document', 'page_no'])['top_px'].diff()
df = df.drop(columns=['top_px'])
df = df.drop(columns=['page_no'])
df['No_of_Special_Char'] = df['text'].str.replace(r"[A-Za-z0-9. (\")(\')(,)(_)(@)]+", "").apply(len)

In [ ]:
len(df)

41910

In [ ]:
from webcolors import *
import numpy as np

In [ ]:
%time
df['color_val'] = np.nan
for x in range(len(df)):
    try:
        df['color_val'][x]  = rgb_to_name(eval(df['color'][x].lstrip('rgba'))[:3])
        print(df['color_val'][x])
        
    except:
        pass

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
black
black
black


/Users/msyeom/anaconda3/envs/pdf/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/msyeom/anaconda3/envs/pdf/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blac

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blac

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blac

...

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blue
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blue
black
black
black
blue
black
b

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blac

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blac

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blac

black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
blac

In [43]:
df

,class,style,id,font-size,vertical-align,color,font-family,font-weight,font-style,position,...,String_Length,No_of_Special_Char,Leading_Char_Upper,Leading_Numeral,Ends_In_Period,Leading_Number_Period,Leading_Char_Period,Roman_Period,Left_Px,LS
12064,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,...,16,0,1,0,0,0,0,0,72,NaN
12065,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,...,40,1,1,0,0,0,0,0,72,27.0
12066,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,...,39,0,1,0,0,0,0,0,72,13.0
12067,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,...,21,3,1,0,0,0,0,0,72,14.0
12068,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,...,13,0,1,0,0,0,1,0,72,13.0
12069,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,...,24,1,1,0,0,0,0,0,72,14.0
12070,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,...,92,3,1,0,0,0,0,0,72,27.0
12071,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,...,34,0,0,0,0,0,0,0,72,13.0
12072,txt,font-size:11px;vertical-align:baseline;color:r...,f2,11px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,...,23,1,1,0,0,0,0,0,72,27.0
12073,txt,font-size:12px;vertical-align:baseline;color:r...,f3,12px,baseline,"rgba(0,0,0,1)",sans-serif,normal,normal,absolute,...,94,2,1,0,0,0,0,0,72,27.0


In [ ]:
df['color_val'].unique()

In [ ]:
df['color_val'].value_counts()

## Export to a csv file.

In [ ]:
df.to_csv('features_v8.csv')

In [73]:
df = df.drop(columns=['page_no'])